<p style="text-align: center"> Escuela Técnica Superior de Ingeniería Informática</p>
<p style="text-align: center">Universidad de Sevilla</p>
<p style="text-align: center">Procesamiento del Lenguaje Natural </p>
<p style="text-align: center"> Tarea 3 </p>
<p> </p>

**Nombre del alumno:**
- Kenny Jesús Flores Huamán

<!-- 
Para generar la tabla de contenidos que se va a haber a continuación, se ha hecho uso del siguiente código
URL : https://stackoverflow.com/questions/21151450/how-can-i-add-a-table-of-contents-to-a-jupyter-jupyterlab-notebook#:~:text=Click%20the%20toc2%20symbol%20in,you%20open%20it%20in%20JupyterLab.

import json
import urllib
def generate_toc(notebook_path, indent_char="&emsp;"):
    is_markdown = lambda it: "markdown" == it["cell_type"]
    is_title = lambda it: it.strip().startswith("#") and it.strip().lstrip("#").lstrip()
    with open(notebook_path, 'r') as in_f:
        nb_json = json.load(in_f)
    for cell in filter(is_markdown, nb_json["cells"]):
        for line in filter(is_title, cell["source"]):
            line = line.strip()
            indent = indent_char * (line.index(" ") - 1)
            title = line.lstrip("#").lstrip()
            url = urllib.parse.quote(title.replace(" ", "-"))
            out_line = f"{indent}[{title}](#{url})<br>\n"
            print(out_line, end="")
            
generate_toc('C6_FloresHuaman.ipynb') -->


# Introduccion 

En esta tarea vamos a hacer uso de la herramienta jupyter notebook para profundizar algunos aspectos del procesamiento del lenguaje natural, en este caso vamos a realizar una predicción de la siguiente palabra que escribamos.


# Librerias importadas

In [1]:
import pandas as pd
import numpy as np
import string
import re
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

2023-04-16 14:56:31.871734: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Cargamos las stopwords en 
# En este caso las stopwords puedan ser útiles
#stopwords = set(stopwords.words('spanish'))

## Dataset utilizado

En esta ocasión, debido a los nulos datasets relacionados con los chats en español, se ha decidido elegir un dataset donde recopilan [+9000 letras de rap en español](https://www.kaggle.com/datasets/smunoz3801/9325-letras-de-rap-en-espaol).



In [64]:
dataset = pd.read_csv("dataset.csv")

Debido a que de todo el dataset lo único que nos interesa es el corpus, vamos a convertirlo en un array numpy

In [96]:
letras = dataset['letra'].values.astype('U')

# Preprocesado

Generamos una máscara para poder eliminar las letras que no tienen letra.

In [97]:
mask = np.char.find(letras, '¿Tienes ya la letra para este tema? Ayúdanos y ¡Envíanosla!') == -1
letras = letras[mask]

In [98]:
letras[0]

'[Denom]\nYo que quería, yo que pedía vida,\nSe partió por la mitad lo que tenía y ahora es frio,\nTe siento dentro mío, como el dolor, como una tenía,\nTragos de colonia, principios de histeria, no hay color,\nYo me elija a no llorar, aunque no sonría,\nTú no sabes que es sufrir, si por amar no perderías todo,\nYo no sé lo que es guardar solo recuerdos y aun así,\nFumo y lo jodo como, todo,\nLuego no vengas llorar, lo escribo solo,\nAunque pensando en ella,\nCon los ojos rojos de llorar y la mano en la polla,\nLos problemas, los collares, mi epopeya,\nNo dejes que me destruya, Freya, ya.\n\n\n[Estribillo: Denom] (x2)\nSi aquí llueve y hiela, como puede ser,\nQue afuera no sientan el frío,\nNunca fui como ellos, yo jamás lo quise ser,\nPerdí mil veces sin saber qué hacer,\nSigo luchando por llenar ese vació,\n\n\n[Ivo Incuerdo]\nSi cae tormenta y truena, como puede ser,\nQue allá afuera no sientan el ruido,\nNunca fui como ellos, jamás serán como yo,\nPara encontrarme, cuantas veces me

In [99]:
def preprocess_text(text):
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)
    text= re.sub(r'[^\w\s]', ' ', text)
    text = text.strip()
    return text
letras = np.char.replace(letras,"\n"," ")
letras = np.char.replace(letras,'\xa0', ' ')
letras = np.vectorize(preprocess_text)(letras)
letras[0]

'Yo que quería  yo que pedía vida  Se partió por la mitad lo que tenía y ahora es frio  Te siento dentro mío  como el dolor  como una tenía  Tragos de colonia  principios de histeria  no hay color  Yo me elija a no llorar  aunque no sonría  Tú no sabes que es sufrir  si por amar no perderías todo  Yo no sé lo que es guardar solo recuerdos y aun así  Fumo y lo jodo como  todo  Luego no vengas llorar  lo escribo solo  Aunque pensando en ella  Con los ojos rojos de llorar y la mano en la polla  Los problemas  los collares  mi epopeya  No dejes que me destruya  Freya  ya      Si aquí llueve y hiela  como puede ser  Que afuera no sientan el frío  Nunca fui como ellos  yo jamás lo quise ser  Perdí mil veces sin saber qué hacer  Sigo luchando por llenar ese vació     Si cae tormenta y truena  como puede ser  Que allá afuera no sientan el ruido  Nunca fui como ellos  jamás serán como yo  Para encontrarme  cuantas veces me he perdido  Sigo luchando por llenar el vació  Como el del estómago  des

# Tokenización

En Keras, tenemos la clase Tokenizer, que nos permite convertir texto en una secuencia de enteros para poder pasarlo a nuestra red neuronal.

Cuando tenemos creado el objeto ```Tokenizer```, se puede usar la función ```fit_on_texts``` para tomar nuestro array de tectos y crea un diccionario para asignarle a cada palabra única en el corpus un número entero.

In [109]:
# Generamos el diccionario y lo guardamos para realizar la función predictiva
tokenizer = Tokenizer()
tokenizer.fit_on_texts(letras)

In [110]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [111]:
tokenizer.word_index

{'que': 1,
 'de': 2,
 'y': 3,
 'el': 4,
 'no': 5,
 'la': 6,
 'en': 7,
 'a': 8,
 'me': 9,
 'es': 10,
 'un': 11,
 'lo': 12,
 'mi': 13,
 'por': 14,
 'con': 15,
 'se': 16,
 'te': 17,
 'los': 18,
 'si': 19,
 'yo': 20,
 'tu': 21,
 'como': 22,
 'las': 23,
 'al': 24,
 'una': 25,
 'pero': 26,
 'del': 27,
 'para': 28,
 'ya': 29,
 'más': 30,
 'sin': 31,
 'soy': 32,
 'cuando': 33,
 'todo': 34,
 'solo': 35,
 'vida': 36,
 'hay': 37,
 'su': 38,
 'mis': 39,
 'porque': 40,
 'ni': 41,
 'ser': 42,
 'o': 43,
 'quiero': 44,
 'esta': 45,
 'tengo': 46,
 'siempre': 47,
 'tú': 48,
 'este': 49,
 'son': 50,
 'pa': 51,
 'nada': 52,
 'cada': 53,
 'le': 54,
 'nos': 55,
 'qué': 56,
 'rap': 57,
 'estoy': 58,
 'mas': 59,
 'tus': 60,
 'sé': 61,
 'nunca': 62,
 'he': 63,
 'tiempo': 64,
 'ahora': 65,
 'eso': 66,
 'mundo': 67,
 'hoy': 68,
 'va': 69,
 'i': 70,
 'así': 71,
 'tan': 72,
 'hasta': 73,
 'aquí': 74,
 'ti': 75,
 'voy': 76,
 'esto': 77,
 'amor': 78,
 'donde': 79,
 'bien': 80,
 'quien': 81,
 'todos': 82,
 'está': 83

In [118]:
sequence_data = tokenizer.texts_to_sequences(letras)

In [133]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

118287


In [120]:
sequences = []
for sequence in sequence_data:
    for i in range(3, len(sequence)):
        words = sequence[i-3:i+1]
        sequences.append(words)
        

In [121]:
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  4373793


array([[   20,     1,   747,    20],
       [    1,   747,    20,     1],
       [  747,    20,     1, 10301],
       [   20,     1, 10301,    36],
       [    1, 10301,    36,    16],
       [10301,    36,    16, 11618],
       [   36,    16, 11618,    14],
       [   16, 11618,    14,     6],
       [11618,    14,     6,   627],
       [   14,     6,   627,    12]])

In [130]:
X, y = map(np.array, zip(*[(seq[0:3], seq[3]) for seq in sequences]))

In [131]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[   20     1   747]
 [    1   747    20]
 [  747    20     1]
 [   20     1 10301]
 [    1 10301    36]
 [10301    36    16]
 [   36    16 11618]
 [   16 11618    14]
 [11618    14     6]
 [   14     6   627]]
Response:  [   20     1 10301    36    16 11618    14     6   627    12]


In [134]:
y = to_categorical(y, num_classes=vocab_size)

MemoryError: Unable to allocate 1.88 TiB for an array with shape (4373793, 118287) and data type float32